Plan :
- intro sur l'article + contexte
- explications / démo de l'implem : alpha, multi classe
- pointeurs vers stats bayésiennes
- explications pour l'extension multiclasse
- lien vers implem

Trouver titre :
- "Getting confidence in classification evaluation metrics"


TODO / Check:
- [ ] autre article : http://oliviercaelen.be/doc/confMatrixBayes_AMAI.pdf
- "multiclass problem"?
- vérifier screenshots (cohérence nombre digits...), inclure la cellule de code ?
- lien sklearn classification report
- lien breast cancer dataset
- regarder les guidelines pour les PR dans sklearn ; sinon soumettre une implem à https://sklearn-evaluation.ploomber.io/ ?
- implem : donner la possibilité de sortir les samples ?
- add a legend for the width of the confidence interval

# What's the problem?

You're tasked with a new classification problem: with the data you're given, you train a model, and get some evaluation metrics (precision, recall, F1 score...). Yay! You can call it a day!

Right?... Right?

But how confident are you in the numbers you have? Would they change, say if you retrained your model, or if your dataset was split differently, or had slightly more or less samples, or had a slightly different class balance? By how much would they move?

Or say you have several models with a similar performance, but trained on different subsets of the data of varying size. Which one would you choose?

We intuitively feel that the size of the testing set plays a large part here. In a similar way you would probably pick the restaurant noted 4.2 ⭐️ on 500+ ratings over the one noted 4.6 ⭐️ on 7 ratings.

So, how can we be confident in the performance of the model we just trained?

# Getting confidence in classification evaluation metrics

A few weeks ago I stumbled upon the paper __["A Probabilistic Interpretation of Precision, Recall and F-Score, with Implication for Evaluation"](https://scholar.google.fr/scholar?q=A+Probabilistic+Interpretation+of+Precision,+Recall+and+F-Score,+with+Implication+for+Evaluation&hl=fr&as_sdt=0&as_vis=1&oi=scholart)__ (Cyril Goutte & Eric Gaussier, 2005). After quickly reading the summary, I got quite excited, because it presents a principled way to compute confidence intervals for evaluation metrics for classification such as precision, recall, or f-score.

Until then, I knew only of two ways to compute confidence intervals for those metrics on a scikit-learn-style classification report or on a precision-recall curve.
The first one is recomputing the full learning-and-predicting loop say several dozens times to get decent distributions for the evaluation metrics of interest, which can be very costly.
The second one is by computing bootstrap estimates, which is very fast, but felt somewhat unsatisfactory, and is in fact not recommended (see __["Statistical inference in retrieval effectiveness evaluation](https://scholar.google.fr/scholar?hl=fr&as_sdt=0%2C5&q=Statistical+inference+in+retrieval+effectiveness+evaluation&btnG=)__ (Savoy, 1997), cited in the paper).

This paper introduces:
- a principled, probabilistic formalism for obtaining posterior distributions on evaluation metrics for binary classification problems;
- a framework to compare the performance of several classifiers on different datasets of the same source;
- a refined framework to compare the performance of two classifiers on the same dataset.

I eagerly read the paper, wanting to know more about the developed formalism, and to see if it would be easy to implement it in Python. I'm indeed regularly using the output of the scikit-learn classification report and sometimes meet questions by colleagues about the confidence we can get in some results based on small data. Answers are usually uneasy variations of "we cannot be very confident until we have more data".

So I'm happy to say that the paper is very clear and readable, and that I quickly implemented a Python function derived from the __[famous scikit-learn classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html#sklearn.metrics.classification_report)__, and then extended it to multiclass problems. I can know report full distributions instead of point estimates, and so can you!

## Implementation and demo

This is the result of the function on the __["breast cancer" dataset]()__, a binary problem, with a linear SVC:

<img src="/figures/evaluation_metrics_with_confidence_binary.png" title="Classification report with confidence intervals." style="width:600px"/>

And this is the result on the __["digits" dataset]()__, a 10-class problem, with a linear SVC:

<img src="/figures/evaluation_metrics_with_confidence_10_classes.png" title="Classification report with confidence intervals for a 10-class problem." style="width:600px"/>

It is possible to pass a parameter `alpha` to change the width of the confidence interval (95% by default, here 90%):

<img src="/figures/evaluation_metrics_with_confidence_binary_alpha_90.png" title="Classification report with confidence intervals." style="width:600px"/>

You can also visualize this with boxplots:

<img src="/figures/10_classes_performance_boxplots.png" title="Visualizing the classification report with boxplots." style="width:600px"/>

//// En fait pas certain qu'il faut ajouter ça, ça rend l'article hyper dense

But wait, there's more! Since you can simulate a big number of classifier performance metrics, you can also visualize the confidence interval on more complicated stuff, such as the ROC curve or the precision-recall plot!

## What's in the paper

Explications sur le principe :
- dans un cas binaire, la précision et le rappel peuvent être naturellement modélisées par un processus binomial
- priors naturels : loi Beta ; comme c'est un prior conjugué, on peut directement récupérer des intervalles de confiance sur p et r
- pour f1 plus compliqué, mais je le fais par simulation : sample des distributions posterior de p et r, calcul des samples de f1, puis métriques sur la distribution obtenue

## Multiclass extension

Explications multiclasse :
- schéma pour montrer comment on passe de l'un à l'autre
- est-ce que c'est vraiment directement du binaire ? pas besoin de modifier le formalisme ?